In [1]:
import os
from itertools import combinations
from IPython.display import HTML
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import r2_score as r2
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from nltk.metrics.agreement import AnnotationTask

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def alpha_df(df, good_only=False):
    data = []
    questions = df.columns
    if good_only: questions = list(set(questions)-set({i for i in questions if len(answers[i].dropna().unique()) > 3}))
    for annotator in df.index:
        for question in questions:
            cell = df[question][annotator]
            if cell is not None : data.append((annotator, question, cell))
    try: return AnnotationTask(data=data).alpha()
    except ZeroDivisionError: return 1

In [4]:
def get_trust_coefs(df, good_only=False):
    alphas = dict()
    for i in range(2, len(df.index)+1):
        for combination in combinations(df.index, i):
            alphas[combination] = alpha_df(df.loc[list(combination)], good_only)
    coefs = dict()
    coef = 1
    sorted_keys = [key for key, val in sorted(alphas.items(), key = lambda ele: ele[1])]
    for i, key in enumerate(sorted_keys):
        if alphas[key] > alphas[sorted_keys[i-1]] : coef += 1
        for annotator in key:
            try: coefs[annotator] += coef*alphas[key]
            except KeyError: coefs[annotator] = coef*alphas[key]
    return {annotator : c/max(coefs.values()) for annotator, c in coefs.items()}

In [5]:
def ident_bad_annot(coefs, good_only=False):
    for annotator, coef in coefs.items():
        if coef <= 0.5: yield annotator

In [6]:
def get_profiles(answers, features, verbose=False):
    profiles, reports = {}, {}
    for annotator in answers.index:
        y = answers.loc[annotator].dropna()
        x = features.loc[y.index]
        X_train, X_test, Y_train, Y_test = train_test_split(x, y, random_state=38)
        profiles[annotator] = make_pipeline(StandardScaler(), SGDRegressor(penalty="elasticnet", validation_fraction=0.42, random_state=24, power_t=0.55, n_iter_no_change=9, loss='huber', learning_rate='invscaling', l1_ratio=0.76, eta0=7.61, epsilon=0.081, early_stopping=True)).fit(X_train, Y_train)
        reports[annotator] = r2(Y_test, profiles[annotator].predict(X_test))
        if verbose: print(annotator, reports[annotator])
    return profiles

In [7]:
def profiles_to_plots(profiles, columns, path):
    for annotator, profile in profiles.items():
        sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'})
        coefs = profile["sgdregressor"].coef_
        n_coefs = coefs.shape
        if len(n_coefs) == 1 or n_coefs[0] == 1:
            fig, axes = plt.subplots(1, 1, figsize=(10, 10))
            if len(n_coefs) == 1: sns.barplot(ax=axes, y=columns, x=coefs, palette="hls")
            else: sns.barplot(ax=axes, y=columns, x=coefs[0], palette="hls")
        else:
            fig, axes = plt.subplots(n_coefs[0], 1, figsize=(10, 30))
            for i, coef in enumerate(coefs):
                sns.barplot(ax=axes[i], y=columns, x=coef, palette="hls")
                axes[i].set_title("label " + str(profile.classes_[i]))
        fig.savefig(path+"/"+annotator +".png")

In [8]:
answers = pd.read_csv("answers.csv", names = ["date_time", "email", "ling", "coref"]+list(range(1, 131))+["comments"])
answers.drop(0, axis=0, inplace=True)
answers.drop("date_time", axis=1, inplace=True)
answers = answers.groupby("email").last()
info, answers = answers[["ling", "coref"]], answers[range(1, 131)]

In [9]:
doubles = {23:128, 38:129, 75:130}
checks = {}
for annotator in answers.index:
    check = []
    for k, v in doubles.items():
        first, last = answers[k][annotator], answers[v][annotator]
        check.append("non vérifié") if None in (first, last) else check.append("approuvé") if first == last else check.append("désapprouvé: "+first+" vs "+last)
    checks[annotator] = check
pd.DataFrame(checks).T.style.applymap(lambda x: "background-color: honeydew" if x == "approuvé" else "background-color: white" if x == "non vérifié" else "background-color: lavenderblush")

,0,1,2
Frederique.bordignon@enpc.fr,non vérifié,non vérifié,non vérifié
anais.halftermeyer@univ-orleans.fr,non vérifié,non vérifié,non vérifié
bergler@cse.concordia.ca,non vérifié,non vérifié,non vérifié
emmanuel.schang@univ-orleans.fr,non vérifié,non vérifié,non vérifié
fanny.krimou@hotmail.fr,non vérifié,non vérifié,non vérifié
goudjoasc@aol.com,non vérifié,non vérifié,non vérifié
ilaine.wang@inalco.fr,désapprouvé: 5 vs 6,désapprouvé: 7 vs 6,approuvé
lisa@kolbasa.fyr,non vérifié,non vérifié,non vérifié
loic.grobol@gmail.com,non vérifié,non vérifié,non vérifié
mariya,approuvé,approuvé,approuvé


In [11]:
answers.drop([128, 129, 130], axis=1, inplace=True)
smaller = {"2" : "1", "3" : "2", "4" : "2", "5" : "3", "6" : "3", "7" : "4"}
gravity = {"0":"grave", "1":"grave", "2":"grave", "3":"pas grave", "4":"pas grave", "5":"pas grave", "6":"pas grave", "7":"pas de fautes"}
centrality = {"0":"fondamental", "1":"périphérique", "2":"périphérique", "3":"fondamental", "4":"fondamental", "5":"périphérique", "6":"périphérique", "7":"pas de fautes"}
small, gravity_eval, centrality_eval = [answers.applymap(lambda x : scale[x] if x in scale.keys() else x) for scale in [smaller, gravity, centrality]]
scale_names = ["standard", "réduit","gravité", "centralité"]
alphas_krip = {"échelle standard": [alpha_df(answers), alpha_df(answers, good_only=True)],
              "échelle réduite": [alpha_df(small), alpha_df(small, good_only=True)],
              "échelle de gravité": [alpha_df(gravity_eval), alpha_df(gravity_eval, good_only=True)],
              "échelle de centralité": [alpha_df(centrality_eval), alpha_df(centrality_eval, good_only=True)]}
print("Alphas de Krippendorff:")
pd.DataFrame.from_dict(alphas_krip, orient="index", columns=["toutes les exemples", "sans mauvaises exemples"])

Alphas de Krippendorff:


,toutes les exemples,sans mauvaises exemples
échelle standard,0.116444,0.252319
échelle réduite,0.169048,0.343351
échelle de gravité,0.247672,0.485453
échelle de centralité,0.169100,0.346860


In [13]:
trust_coefs = {'standard_coef':get_trust_coefs(answers),
               'standard_sans_mauvaises_coef':get_trust_coefs(answers, good_only=True),
               'réduit_coef':get_trust_coefs(small),
               'réduit_sans_mauvaises_coef':get_trust_coefs(small, good_only=True),
               'gravité_coef':get_trust_coefs(gravity_eval),
               'gravité_sans_mauvaises_coef':get_trust_coefs(gravity_eval, good_only=True),
               'centralité_coef':get_trust_coefs(centrality_eval),
               'centralité_sans_mauvaises_coef':get_trust_coefs(centrality_eval, good_only=True)}
bad_annot = {scale:list(ident_bad_annot(annot)) for scale, annot in trust_coefs.items()}
if not(any(bad_annot.values())): print("Il n'y a pas d'annotateurs douteaux.")
else:
    print("Annotateurs douteux:")
    display(pd.DataFrame({scale:["❌" if annotator in annot else None for annotator in answers.index] for scale, annot in bad_annot.items()}, index=answers.index).dropna(how="all", axis=0).dropna(how="all", axis=1).fillna(""))

Annotateurs douteux:


,standard_sans_mauvaises_coef,centralité_sans_mauvaises_coef
email,,
ilaine.wang@inalco.fr,,❌
mariya,❌,


In [14]:
alphas_krip = {"échelle standard": [alpha_df(answers[~answers.index.isin(bad_annot["standard_coef"])]), alpha_df(answers[~answers.index.isin(bad_annot["standard_sans_mauvaises_coef"])], good_only=True)],
              "échelle réduite": [alpha_df(small[~answers.index.isin(bad_annot["réduit_coef"])]), alpha_df(small[~answers.index.isin(bad_annot["réduit_sans_mauvaises_coef"])], good_only=True)],
              "échelle de gravité": [alpha_df(gravity_eval[~answers.index.isin(bad_annot["gravité_coef"])]), alpha_df(gravity_eval[~answers.index.isin(bad_annot["gravité_sans_mauvaises_coef"])], good_only=True)],
              "échelle de centralité": [alpha_df(centrality_eval[~answers.index.isin(bad_annot["centralité_coef"])]), alpha_df(centrality_eval[~answers.index.isin(bad_annot["centralité_sans_mauvaises_coef"])], good_only=True)]}
print("Alphas de Krippendorff sans mauvaises annotateurs:")
pd.DataFrame.from_dict(alphas_krip, orient="index", columns=["toutes les exemples", "sans mauvaises exemples"])

Alphas de Krippendorff sans mauvaises annotateurs:


,toutes les exemples,sans mauvaises exemples
échelle standard,0.116444,0.274267
échelle réduite,0.169048,0.343351
échelle de gravité,0.247672,0.485453
échelle de centralité,0.169100,0.423513


In [16]:
info1 = pd.concat([info, pd.DataFrame(trust_coefs)], axis=1, join="inner").round(3)
info1.insert(loc=2, column='average', value=info1.loc[:, ~info1.columns.isin(['ling', 'coref'])].mean(axis=1).round(3))
display(info1.sort_values(by="average", ascending=False).style.set_properties(**{'background-color': 'floralwhite'}, subset=['average']))

,ling,coref,average,standard_coef,standard_sans_mauvaises_coef,réduit_coef,réduit_sans_mauvaises_coef,gravité_coef,gravité_sans_mauvaises_coef,centralité_coef,centralité_sans_mauvaises_coef
sylvie.billot@univ-orleans.fr,non,oui,0.884000,0.984000,0.709000,0.938000,0.839000,1.000000,1.000000,0.887000,0.712000
fanny.krimou@hotmail.fr,oui,non,0.851000,0.829000,0.936000,0.731000,0.934000,0.727000,0.961000,0.721000,0.968000
ygor.gallina@univ-nantes.fr,non,non,0.851000,0.816000,0.936000,0.750000,0.934000,0.702000,0.961000,0.743000,0.968000
Frederique.bordignon@enpc.fr,oui,non,0.835000,0.768000,1.000000,0.667000,1.000000,0.634000,0.990000,0.624000,1.000000
emmanuel.schang@univ-orleans.fr,oui,oui,0.834000,0.799000,0.936000,0.691000,0.934000,0.746000,0.961000,0.633000,0.968000
bergler@cse.concordia.ca,non,oui,0.824000,0.769000,0.936000,0.685000,0.934000,0.656000,0.961000,0.683000,0.968000
lisa@kolbasa.fyr,oui,oui,0.816000,0.754000,0.906000,0.675000,0.912000,0.638000,0.935000,0.727000,0.979000
anais.halftermeyer@univ-orleans.fr,oui,oui,0.813000,0.736000,0.736000,0.782000,0.950000,0.700000,0.997000,0.748000,0.858000
loic.grobol@gmail.com,oui,oui,0.807000,0.984000,0.856000,0.719000,0.810000,0.705000,0.920000,0.749000,0.717000
ilaine.wang@inalco.fr,oui,non,0.783000,1.000000,0.803000,0.833000,0.640000,0.951000,0.903000,0.686000,0.448000


In [17]:
good_index = list(set(answers.columns)-{i for i in answers.columns if len(answers[i].dropna().unique()) > 3})
metrics = pd.read_csv("scores.csv", index_col="id").drop("Unnamed: 0", axis=1).drop("name", axis=1)
recalls = metrics.drop("MUC_P", axis=1).drop("MUC_F1", axis=1).drop("B3_P", axis=1).drop("B3_F1", axis=1).drop("CEAFm_P", axis=1).drop("CEAFm_F1", axis=1).drop("CEAFe_P", axis=1).drop("CEAFe_F1", axis=1).drop("BLANC_P", axis=1).drop("BLANC_F1", axis=1).drop("LEA_P", axis=1).drop("LEA_F1", axis=1).drop("CoNLL_2012", axis=1).T
precisions = metrics.drop("MUC_R", axis=1).drop("MUC_F1", axis=1).drop("B3_R", axis=1).drop("B3_F1", axis=1).drop("CEAFm_R", axis=1).drop("CEAFm_F1", axis=1).drop("CEAFe_R", axis=1).drop("CEAFe_F1", axis=1).drop("BLANC_R", axis=1).drop("BLANC_F1", axis=1).drop("LEA_R", axis=1).drop("LEA_F1", axis=1).drop("CoNLL_2012", axis=1).T
fs = metrics.drop("MUC_R", axis=1).drop("MUC_P", axis=1).drop("B3_R", axis=1).drop("B3_P", axis=1).drop("CEAFm_R", axis=1).drop("CEAFm_P", axis=1).drop("CEAFe_R", axis=1).drop("CEAFe_P", axis=1).drop("BLANC_R", axis=1).drop("BLANC_P", axis=1).drop("LEA_R", axis=1).drop("LEA_P", axis=1).T
a_mean = answers[good_index].astype(float).mean()
a_mode = answers[good_index].astype(float).mode().mean()
s_mean = small[good_index].astype(float).mean()*1.75
s_mode = small[good_index].astype(float).mode().mean()*1.75
gravity_to_num = {"grave":0., "pas grave":1., "pas de fautes":2.}
g_mean = gravity_eval.applymap(lambda x : gravity_to_num[x] if x in gravity_to_num.keys() else x).mean()*3.5
g_mode = gravity_eval.applymap(lambda x : gravity_to_num[x] if x in gravity_to_num.keys() else x).mode().mean()*3.5
average = {"a_mean":a_mean, "a_mode":a_mode, "s_mean":s_mean, "s_mode":s_mode, "g_mean":g_mean, "g_mode":g_mode}
bars = {"moyenne":{i:pd.Series([a_mean[i], s_mean[i], g_mean[i], fs[i]["MUC_F1"]*7, fs[i]["B3_F1"]*7, fs[i]["CEAFm_F1"]*7, fs[i]["CEAFe_F1"]*7, fs[i]["BLANC_F1"]*7, fs[i]["LEA_F1"]*7, fs[i]["CoNLL_2012"]*7], index=["standard", "réduit", "gravité_moyenne", "MUC", "B_CUBED", "CEAFm", "CEAFe", "BLANC", "LEA", "CoNLL"]) for i in good_index}, "mode":{i:pd.Series([a_mode[i], s_mode[i], g_mode[i], fs[i]["MUC_F1"]*7, fs[i]["B3_F1"]*7, fs[i]["CEAFm_F1"]*7, fs[i]["CEAFe_F1"]*7, fs[i]["BLANC_F1"]*7, fs[i]["LEA_F1"]*7, fs[i]["CoNLL_2012"]*7], index=["standard", "réduit", "gravité_moyenne", "MUC", "B_CUBED", "CEAFm", "CEAFe", "BLANC", "LEA", "CoNLL"]) for i in good_index}}


head = "<table><tbody>"
for bar in bars:
    row = "<tr><th>{}</th>".format(bar)
    for name, series in bars[bar].items():
        s = series.copy()
        s.name=name
        row += "<td>{}</td>".format(s.to_frame().style.bar(align="mid", color='#5fba7d').render())
    row += '</tr>'
    head += row

head+= "</tbody></table>"

print("Comparaison naive (de 0 à 7, des échelles plus petites sont multipliés par des coefficients pour obtenir 7 comme maximum):")
HTML(head)

Comparaison naive (de 0 à 7, des échelles plus petites sont multipliés par des coefficients pour obtenir 7 comme maximum):


In [19]:
differences = {metric : {a_name : fs[good_index].loc[metric]-a/7 for a_name, a in average.items()} for metric in fs.index}
differences_abs = {metric: {a_name : (fs[good_index].loc[metric]-a/7).abs() for a_name, a in average.items()} for metric in fs.index}
differences_pos = {metric : {name : difference[difference > 0] for name, difference in users.items()} for metric, users in differences.items()}
differences_neg = {metric : {name : difference[difference < 0] for name, difference in users.items()} for metric, users in differences.items()}
means = {metric : {name : difference.mean() for name, difference in users.items()} for metric, users in differences.items()}
means_abs = {metric : {name : difference.mean() for name, difference in users.items()} for metric, users in differences_abs.items()}
means_pos = {metric : {name : difference.mean() for name, difference in users.items()} for metric, users in differences_pos.items()}
means_neg = {metric : {name : difference.mean() for name, difference in users.items()  if not(difference.empty)} for metric, users in differences_neg.items()}

In [20]:
pd.DataFrame.from_dict(means)

,MUC_F1,B3_F1,CEAFm_F1,CEAFe_F1,BLANC_F1,LEA_F1,CoNLL_2012
a_mean,0.284579,0.319264,0.306191,0.260156,0.275412,0.219394,0.288000
a_mode,0.278564,0.313249,0.300176,0.254141,0.269397,0.213379,0.281985
s_mean,0.309976,0.344661,0.331588,0.285553,0.300809,0.244791,0.313397
s_mode,0.304257,0.338942,0.325869,0.279834,0.295090,0.239072,0.307678
g_mean,0.428172,0.462857,0.449784,0.403749,0.419005,0.362987,0.431593
g_mode,0.421873,0.456558,0.443485,0.397450,0.412706,0.356688,0.425294


In [22]:
pd.DataFrame.from_dict(means_abs)

,MUC_F1,B3_F1,CEAFm_F1,CEAFe_F1,BLANC_F1,LEA_F1,CoNLL_2012
a_mean,0.289401,0.321391,0.308325,0.269343,0.280678,0.230573,0.291019
a_mode,0.293603,0.325976,0.313205,0.282663,0.285004,0.240331,0.298520
s_mean,0.313668,0.346110,0.333044,0.290440,0.304945,0.253331,0.315443
s_mode,0.312454,0.346658,0.333262,0.291877,0.304330,0.254851,0.316430
g_mean,0.430282,0.462857,0.449784,0.405354,0.421559,0.368196,0.432057
g_mode,0.430070,0.464274,0.450878,0.407794,0.421946,0.369135,0.434046


In [24]:
pd.DataFrame.from_dict(means_pos)

,MUC_F1,B3_F1,CEAFm_F1,CEAFe_F1,BLANC_F1,LEA_F1,CoNLL_2012
a_mean,0.302296,0.324434,0.311197,0.290489,0.289021,0.250334,0.297029
a_mode,0.327545,0.355625,0.341247,0.326212,0.317375,0.271539,0.327571
s_mean,0.328453,0.349814,0.336576,0.303356,0.314833,0.269532,0.318451
s_mode,0.333700,0.361083,0.347143,0.313647,0.319961,0.274788,0.328697
g_mean,0.452119,0.462857,0.449784,0.415059,0.436872,0.380023,0.437361
g_mode,0.460983,0.484972,0.471032,0.424095,0.445524,0.387433,0.452586


In [26]:
pd.DataFrame.from_dict(means_neg)

,MUC_F1,B3_F1,CEAFm_F1,CEAFe_F1,BLANC_F1,LEA_F1,CoNLL_2012
a_mean,-0.063492,-0.084027,-0.084286,-0.051840,-0.069340,-0.055198,-0.059634
a_mode,-0.074254,-0.062837,-0.064330,-0.080473,-0.068497,-0.088716,-0.072573
s_mean,-0.072917,-0.057241,-0.057500,-0.048255,-0.054459,-0.067465,-0.080809
s_mode,-0.080945,-0.076194,-0.073005,-0.079282,-0.091245,-0.089035,-0.086427
g_mean,-0.083333,NaN,NaN,-0.031695,-0.033626,-0.068582,-0.018309
g_mode,-0.080945,-0.076194,-0.073005,-0.102141,-0.091245,-0.122916,-0.086427


In [28]:
%%capture
errors = pd.read_csv("errors.csv", sep=";", index_col="id", header=0, names = ["id", "frontiers", "type", "bruit", "silence", "non-pertinentes", "absence"])
profiles = {"STANDARD\_": get_profiles(answers, errors, verbose=True), "REDUCED_": get_profiles(small, errors, verbose=True), "GRAVITY_": get_profiles(gravity_eval.applymap(lambda x : gravity_to_num[x] if x in gravity_to_num.keys() else x), errors, verbose=True), "": get_profiles(fs, errors, verbose=True)}
avg_profiles = {"a_mean":get_profiles(a_mean.to_frame(name="a_mean").T, errors, verbose=True), "a_mode":get_profiles(a_mode.to_frame(name="a_mode").T, errors, verbose=True), "s_mean":get_profiles(s_mean.to_frame(name="s_mean").T, errors, verbose=True), "s_mode":get_profiles(s_mode.to_frame(name="s_mode").T, errors, verbose=True), "g_mean":get_profiles(g_mean.to_frame(name="g_mean").T, errors, verbose=True), "g_mode":get_profiles(g_mode.to_frame(name="g_mode").T, errors, verbose=True)}
profiles["STANDARD\_"].update(avg_profiles["a_mean"])
profiles["STANDARD\_"].update(avg_profiles["a_mode"])
profiles["REDUCED\_"].update(avg_profiles["s_mean"])
profiles["REDUCED\_"].update(avg_profiles["s_mode"])
profiles["GRAVITY\_"].update(avg_profiles["g_mean"])
profiles["GRAVITY\_"].update(avg_profiles["g_mode"])
for name, profile in profiles.items():
    path = "profiles/"+name
    if not(os.path.isdir(path)): os.mkdir(path)
    profiles_to_plots(profile, errors.columns, path)

In [29]:
anonymes = {annotator : "A"+str(i) for annotator, i in zip(answers.index, list(range(1, len(answers.index)+1)))}
anonymes.update({f : f for f in fs.index})
anonymes.update({"a_mean":"MEAN", "s_mean":"MEAN", "g_mean":"MEAN", "a_mode":"MODE", "s_mode":"MODE", "g_mode":"MODE"})
c = {"STANDARD\_":1/7, "REDUCED\_":4/7, "GRAVITY\_":2/7, "":1}

In [30]:
results = pd.DataFrame.from_dict({scale+anonymes[annotator]: c[scale]*model["sgdregressor"].coef_ for scale, profile in profiles.items() for annotator, model in profile.items()}, orient="index", columns=errors.columns)

In [31]:
frontiers = results["frontiers"].sort_values(ascending=False).to_frame()
frontiers.insert(0, "name", frontiers.index)
frontiers.style.hide_index()

name,frontiers
REDUCED\_A6,-0.033000
GRAVITY\_A10,-0.057000
STANDARD\_A7,-0.097000
GRAVITY\_A7,-0.118000
GRAVITY\_A11,-0.152000
STANDARD\_A10,-0.166000
BLANC_F1,-0.174000
STANDARD\_A6,-0.182000
STANDARD\_A8,-0.212000
MUC_F1,-0.242000


In [33]:
type_ = results["type"].sort_values(ascending=False).to_frame()
type_.insert(0, "name", type_.index)
type_.style.hide_index()

name,type
B3_F1,-0.150000
CoNLL_2012,-0.217000
LEA_F1,-0.220000
GRAVITY\_A7,-0.239000
GRAVITY\_A8,-0.240000
MUC_F1,-0.249000
CEAFe_F1,-0.283000
STANDARD\_A10,-0.310000
CEAFm_F1,-0.340000
STANDARD\_A12,-0.362000


In [35]:
bruit = results["bruit"].sort_values(ascending=False).to_frame()
bruit.insert(0, "name", bruit.index)
bruit.style.hide_index()

name,bruit
CoNLL_2012,-0.083000
GRAVITY\_A7,-0.105000
MUC_F1,-0.121000
REDUCED\_A1,-0.122000
CEAFm_F1,-0.139000
STANDARD\_A9,-0.146000
GRAVITY\_A11,-0.167000
LEA_F1,-0.207000
BLANC_F1,-0.233000
STANDARD\_A7,-0.316000


In [37]:
silence = results["silence"].sort_values(ascending=False).to_frame()
silence.insert(0, "name", silence.index)
silence.style.hide_index()

name,silence
GRAVITY\_A6,-0.011000
STANDARD\_A12,-0.262000
GRAVITY\_A8,-0.265000
STANDARD\_A11,-0.274000
STANDARD\_A9,-0.329000
GRAVITY\_A11,-0.391000
STANDARD\_A10,-0.406000
CEAFm_F1,-0.408000
CEAFe_F1,-0.447000
GRAVITY\_A1,-0.490000


In [39]:
non_pertinentes = results["non-pertinentes"].sort_values(ascending=False).to_frame()
non_pertinentes.insert(0, "name", non_pertinentes.index)
non_pertinentes.style.hide_index()

name,non-pertinentes
GRAVITY\_A6,-0.018000
BLANC_F1,-0.074000
STANDARD\_A11,-0.080000
REDUCED\_MEAN,-0.097000
GRAVITY\_A11,-0.164000
STANDARD\_A4,-0.167000
CoNLL_2012,-0.179000
CEAFe_F1,-0.222000
GRAVITY\_MEAN,-0.319000
STANDARD\_A8,-0.335000


In [41]:
absence = results["absence"].sort_values(ascending=False).to_frame()
absence.insert(0, "name", absence.index)
absence.style.hide_index()

name,absence
MUC_F1,-0.076000
STANDARD\_A10,-0.099000
CEAFm_F1,-0.101000
GRAVITY\_A11,-0.124000
STANDARD\_A7,-0.170000
CoNLL_2012,-0.187000
GRAVITY\_A7,-0.247000
B3_F1,-0.264000
GRAVITY\_A8,-0.265000
STANDARD\_A6,-0.283000
